# Volatility Modeling - Part 1
## Recap, Stylized Facts, EWMA and GARCH Models

This notebook covers:
1. **Volatility Modeling Recap**: EWMA, GARCH and related models
2. **Stylized Facts**: Key properties of financial volatility
3. **EWMA Model**: Exponentially Weighted Moving Average
4. **GARCH Models**: Manual optimization and arch package implementation
5. **Realized Volatility**: Comparison with model-based estimates

In [ ]:
# !pip install arch yfinance wrds

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from arch import arch_model
import zipfile
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## 1. Data Loading

### S&P 500 Returns from WRDS

We'll use WRDS (Wharton Research Data Services) to access high-quality financial data.

In [ ]:
# Connect to WRDS and download S&P 500 data
import wrds

# Connect to WRDS (you might have to confirm with 2FA)
db = wrds.Connection(wrds_username=None)

# Download S&P 500 index data from Compustat
# Date range: 2000-01-03 to 2022-06-28 (matching RV5 data)
query = """
SELECT datadate as date, prccd as price
FROM comp.idx_daily
WHERE gvkeyx = '000003'
  AND datadate >= '2000-01-03' 
  AND datadate <= '2022-06-28'
ORDER BY datadate
"""

print("Downloading S&P 500 data from WRDS/Compustat...")
data = db.raw_sql(query)
db.close()

# Process data
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')

# Calculate log returns (in percentage)
returns = 100 * np.log(data['price'] / data['price'].shift(1))
returns = returns.dropna()

print(f"\nData loaded: {len(returns)} observations")
print(f"Date range: {returns.index[0]} to {returns.index[-1]}")
print(f"\nBasic statistics:")
print(returns.describe())

In [ ]:
# # Alternative: Using yfinance (uncomment to use)
# import yfinance as yf

# # Download S&P 500 data
# print("Downloading S&P 500 data from Yahoo Finance...")
# sp500 = yf.download('^GSPC', start='2000-01-01', end='2024-12-31', progress=False)

# # Calculate log returns (in percentage)
# returns = 100 * np.log(sp500['Close'] / sp500['Close'].shift(1))
# returns = returns.dropna()
# returns.name = 'returns'
# returns = returns['^GSPC']

# print(f"\nData loaded: {len(returns)} observations")
# print(f"Date range: {returns.index[0]} to {returns.index[-1]}")
# print(f"\nBasic statistics:")
# print(returns.describe())

## 2. Stylized Facts of Volatility

Financial volatility exhibits several well-documented empirical regularities:

1. **Volatility Clustering**: High (low) volatility tends to be followed by high (low) volatility
2. **Fat Tails**: Return distributions have heavier tails than the normal distribution
3. **Leverage Effect**: Negative returns are associated with larger increases in volatility than positive returns
4. **Mean Reversion**: Volatility tends to revert to a long-run mean
5. **Long Memory**: Volatility shocks persist for extended periods

In [ ]:
# Visualize returns and distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Time series plot
axes[0, 0].plot(returns.index, returns.values, linewidth=0.5, color='navy', alpha=0.7)
axes[0, 0].set_title('S&P 500 Daily Returns', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Return (%)')
axes[0, 0].axhline(y=0, color='red', linestyle='--', linewidth=0.5)
axes[0, 0].grid(True, alpha=0.3)

# Histogram with normal overlay
axes[0, 1].hist(returns.values, bins=100, alpha=0.7, color='navy', density=True, edgecolor='black')
mu, std = returns.mean(), returns.std()
x = np.linspace(returns.min(), returns.max(), 100)
axes[0, 1].plot(x, stats.norm.pdf(x, mu, std), 'r-', linewidth=2, label='Normal Distribution')
axes[0, 1].set_title('Distribution of Returns', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Return (%)')
axes[0, 1].set_ylabel('Density')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Absolute returns (proxy for volatility)
abs_returns = np.abs(returns)
axes[1, 0].plot(abs_returns.index, abs_returns.values, linewidth=0.5, color='darkred', alpha=0.7)
axes[1, 0].set_title('Absolute Returns (Volatility Proxy)', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('|Return| (%)')
axes[1, 0].set_xlabel('Date')
axes[1, 0].grid(True, alpha=0.3)

# ACF of absolute returns (volatility clustering)
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(abs_returns.values, lags=50, ax=axes[1, 1], alpha=0.05)
axes[1, 1].set_title('ACF of Absolute Returns', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Lag')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print statistics
print("\n" + "="*60)
print("STYLIZED FACTS ANALYSIS")
print("="*60)
print(f"Mean return: {returns.mean():.4f}%")
print(f"Volatility (std): {returns.std():.4f}%")
print(f"Skewness: {returns.skew():.4f} (normal = 0)")
print(f"Excess Kurtosis: {returns.kurtosis():.4f} (normal = 0)")
print(f"\n✓ Fat tails confirmed: Excess kurtosis = {returns.kurtosis():.2f} >> 0")
print(f"✓ Volatility clustering visible in absolute returns plot")

## 3. EWMA (Exponentially Weighted Moving Average)

### Simple Volatility Model

The EWMA model is a simple approach for estimating volatility that gives more weight to recent observations:

$$\sigma_t^2 = \lambda \sigma_{t-1}^2 + (1-\lambda) r_{t-1}^2$$

where:
- $\lambda$: Decay factor (typically 0.94 for daily data, RiskMetrics uses 0.94)
- $\sigma_t^2$: Conditional variance at time t
- $r_t$: Return at time t

**Key Properties:**
- Simple recursive formula
- No parameters to estimate (λ often fixed)
- More weight on recent observations
- Special case of IGARCH (Integrated GARCH) with $\alpha + \beta = 1$

**RiskMetrics (J.P. Morgan):**
- Popularized EWMA for VaR calculations
- Recommended $\lambda = 0.94$ for daily data
- Recommended $\lambda = 0.97$ for monthly data

In [ ]:
from arch.univariate import EWMAVariance, ConstantMean

mdl = ConstantMean(returns, volatility=EWMAVariance(0.94))
ewma_fit = mdl.fit()
ewma_fit.summary()
ewma_vol = ewma_fit.conditional_volatility

In [ ]:
# Compute fitted volatilities using EWMA parameters
T = len(returns)
sigma2_manual = np.zeros(T)
sigma2_manual[0] = returns.var()
for t in range(1, T):
    sigma2_manual[t] = 0.06 * (returns.iloc[t-1])**2 + 0.94 * sigma2_manual[t-1]

ewma_vol_manual = pd.Series(np.sqrt(sigma2_manual), index=returns.index, name='ewma_manual')
ewma_vol_manual

## 4. GARCH Models

### GARCH(1,1) Model

The GARCH(1,1) model specifies conditional variance as:

$$\sigma_t^2 = \omega + \alpha r_{t-1}^2 + \beta \sigma_{t-1}^2$$

where:
- $\omega > 0$: Long-run variance component
- $\alpha \geq 0$: Reaction to market shocks
- $\beta \geq 0$: Persistence of volatility
- $\alpha + \beta < 1$: Stationarity condition

**Key Properties:**
- Captures volatility clustering
- Mean-reverting to long-run variance: $\sigma_L^2 = \omega / (1 - \alpha - \beta)$
- Persistence: $\alpha + \beta$ (close to 1 indicates high persistence)

**Estimation:**
Maximum Likelihood Estimation (MLE) assuming normal innovations:

$$\log L(\theta) = -\frac{1}{2}\sum_{t=1}^{T}\left[\log(2\pi) + \log(\sigma_t^2) + \frac{r_t^2}{\sigma_t^2}\right]$$

where $\theta = (\omega, \alpha, \beta)$

### Manual GARCH(1,1) Estimation

First, we demonstrate how to estimate GARCH parameters by manually optimizing the log-likelihood function.

In [ ]:
# Manual GARCH(1,1) estimation via log-likelihood optimization
from scipy.optimize import minimize

def garch_log_likelihood(params, returns):
    """
    Compute negative log-likelihood for GARCH(1,1) model
    """
    mu, omega, alpha, beta = params
    T = len(returns)
    
    # Initialize variance
    sigma2 = np.zeros(T)
    sigma2[0] = returns.var()  # Unconditional variance
    
    # Compute conditional variances
    for t in range(1, T):
        sigma2[t] = omega + alpha * (returns.iloc[t-1] - mu)**2 + beta * sigma2[t-1]
    
    # Compute log-likelihood
    log_lik = -0.5 * np.sum(np.log(2 * np.pi) + np.log(sigma2) + (returns-mu)**2 / sigma2)
    
    # Return negative log-likelihood (for minimization)
    return -log_lik

# Initial parameter values
initial_params = [0.01, 0.01, 0.05, 0.90]

# Constraints: omega > 0, alpha >= 0, beta >= 0, alpha + beta < 1
constraints = (
    {'type': 'ineq', 'fun': lambda x: x[1]},  # omega > 0
    {'type': 'ineq', 'fun': lambda x: x[2]},  # alpha >= 0
    {'type': 'ineq', 'fun': lambda x: x[3]},  # beta >= 0
    {'type': 'ineq', 'fun': lambda x: 1 - x[2] - x[3]}  # alpha + beta < 1
)

# Bounds for parameters
bounds = ((None,None), (1e-6, None), (0, 1), (0, 1))

print("Optimizing GARCH(1,1) parameters manually...")
print("Initial parameters: mu= {:.4f}, ω={:.4f}, α={:.4f}, β={:.4f}\n".format(*initial_params))

# Optimize
result = minimize(garch_log_likelihood, initial_params, args=(returns,),
                  method='SLSQP', bounds=bounds, constraints=constraints)

# Extract optimal parameters
mu_manual, omega_manual, alpha_manual, beta_manual = result.x

print("="*60)
print("MANUAL GARCH(1,1) ESTIMATION RESULTS")
print("="*60)
print(f"Optimization successful: {result.success}")
print(f"Log-likelihood: {-result.fun:.2f}")
print(f"\nOptimal parameters:")
print(f"mu:    {mu_manual:.6f}")
print(f"ω (omega):    {omega_manual:.6f}")
print(f"α (alpha):    {alpha_manual:.6f}")
print(f"β (beta):     {beta_manual:.6f}")
print(f"\nPersistence (α+β): {alpha_manual + beta_manual:.6f}")
print(f"Long-run volatility: {np.sqrt(omega_manual / (1 - alpha_manual - beta_manual)):.4f}%")

# Compute fitted volatilities using manual parameters
T = len(returns)
sigma2_manual = np.zeros(T)
sigma2_manual[0] = returns.var()
for t in range(1, T):
    sigma2_manual[t] = omega_manual + alpha_manual * (returns.iloc[t-1]-mu_manual)**2 + beta_manual * sigma2_manual[t-1]

garch_vol_manual = pd.Series(np.sqrt(sigma2_manual), index=returns.index, name='garch_manual')

### Using the `arch` Package

Now we use the professional `arch` package for comparison and to get additional statistics.

## 3. GARCH Models Recap

### GARCH(1,1) Model

The GARCH(1,1) model specifies conditional variance as:

$$\sigma_t^2 = \omega + \alpha r_{t-1}^2 + \beta \sigma_{t-1}^2$$

where:
- $\omega > 0$: Long-run variance component
- $\alpha \geq 0$: Reaction to market shocks
- $\beta \geq 0$: Persistence of volatility
- $\alpha + \beta < 1$: Stationarity condition

**Key Properties:**
- Captures volatility clustering
- Mean-reverting to long-run variance: $\sigma_L^2 = \omega / (1 - \alpha - \beta)$
- Persistence: $\alpha + \beta$ (close to 1 indicates high persistence)

In [ ]:
# Estimate GARCH(1,1) model
print("Estimating GARCH(1,1) model...\n")
model_garch = arch_model(returns, vol='Garch', p=1, q=1, dist='normal')
garch_fit = model_garch.fit(disp='off')

print(garch_fit.summary())

# Extract parameters
omega = garch_fit.params['omega']
alpha = garch_fit.params['alpha[1]']
beta = garch_fit.params['beta[1]']

print(f"\n{'='*60}")
print("GARCH(1,1) PARAMETERS")
print(f"{'='*60}")
print(f"ω (omega):           {omega:.6f}")
print(f"α (alpha):           {alpha:.6f}")
print(f"β (beta):            {beta:.6f}")
print(f"\nPersistence (α+β):   {alpha + beta:.6f}")
print(f"Long-run variance:   {omega / (1 - alpha - beta):.6f}")
print(f"Long-run volatility: {np.sqrt(omega / (1 - alpha - beta)):.4f}%")

# Extract conditional volatility
garch_vol = garch_fit.conditional_volatility

### GJR-GARCH Model - Leverage Effect

The GJR-GARCH model (Glosten, Jagannathan, Runkle, 1993) captures asymmetric volatility:

$$\sigma_t^2 = \omega + \alpha\varepsilon_{t-1}^2 + \gamma\varepsilon_{t-1}^2\mathbb{I}_{\varepsilon_{t-1}<0} + \beta \sigma_{t-1}^2$$

where $\mathbb{I}_{\varepsilon_{t-1}<0}$ is an indicator function for negative shocks.

**Interpretation:**
- $\gamma > 0$: Negative shocks increase volatility more than positive shocks
- Impact of positive shock: $\alpha$
- Impact of negative shock: $\alpha + \gamma$
- This captures the "leverage effect" in equity markets

In [ ]:
# Estimate GJR-GARCH(1,1) model
print("Estimating GJR-GARCH(1,1) model...\n")
model_gjr = arch_model(returns, vol='GARCH', p=1, o=1, q=1, dist='normal')
gjr_fit = model_gjr.fit(disp='off')

print(gjr_fit.summary())

# Extract parameters
omega_gjr = gjr_fit.params['omega']
alpha_gjr = gjr_fit.params['alpha[1]']
gamma_gjr = gjr_fit.params['gamma[1]']
beta_gjr = gjr_fit.params['beta[1]']

print(f"\n{'='*60}")
print("GJR-GARCH PARAMETERS")
print(f"{'='*60}")
print(f"ω (omega):                    {omega_gjr:.6f}")
print(f"α (alpha):                    {alpha_gjr:.6f}")
print(f"γ (gamma):                    {gamma_gjr:.6f}")
print(f"β (beta):                     {beta_gjr:.6f}")
print(f"\nTotal impact (negative):      {alpha_gjr + gamma_gjr:.6f}")
print(f"Asymmetry ratio:              {(alpha_gjr + gamma_gjr) / alpha_gjr:.3f}")

if gamma_gjr > 0:
    print(f"\n✓ Leverage effect confirmed: γ = {gamma_gjr:.6f} > 0")
    print(f"  Negative shocks increase volatility {((alpha_gjr + gamma_gjr) / alpha_gjr - 1) * 100:.1f}% more")

# Extract conditional volatility
gjr_vol = gjr_fit.conditional_volatility

In [ ]:
# Compare GARCH and GJR-GARCH volatility estimates
fig, ax = plt.subplots(figsize=(10, 4))

# Plot both volatility series
ax.plot(garch_vol.index, garch_vol.values, linewidth=1, label='GARCH(1,1)', alpha=0.8)
ax.plot(gjr_vol.index, gjr_vol.values, linewidth=1, label='GJR-GARCH(1,1)', alpha=0.8)
ax.plot(ewma_vol_manual.index, ewma_vol_manual.values, linewidth=1, label='EWMA', alpha=0.8)
ax.set_title('Conditional Volatility: EWMA vs GARCH vs GJR-GARCH', fontsize=12, fontweight='bold')
ax.set_ylabel('Volatility (%)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

...and heavy tails?

In [ ]:
# Estimate GARCH(1,1) model
print("Estimating GARCH(1,1)-t model...\n")
model_garch_t = arch_model(returns, vol='Garch', p=1, q=1, dist='StudentsT')
garch_t_fit = model_garch_t.fit(disp='off')

print(garch_t_fit.summary())

# Extract parameters
omega_t = garch_t_fit.params['omega']
alpha_t = garch_t_fit.params['alpha[1]']
beta_t = garch_t_fit.params['beta[1]']

print(f"\n{'='*60}")
print("GARCH(1,1) PARAMETERS")
print(f"{'='*60}")
print(f"ω (omega):           {omega_t:.6f}")
print(f"α (alpha):           {alpha_t:.6f}")
print(f"β (beta):            {beta_t:.6f}")
print(f"\nPersistence (α+β):   {alpha_t + beta_t:.6f}")
print(f"Long-run variance:   {omega_t / (1 - alpha_t - beta_t):.6f}")
print(f"Long-run volatility: {np.sqrt(omega_t / (1 - alpha_t - beta_t)):.4f}%")

# Extract conditional volatility
garch_vol_t = garch_t_fit.conditional_volatility

# Estimate GJR-GARCH(1,1)-t model
print("Estimating GJR-GARCH(1,1) model...\n")
model_gjr_t = arch_model(returns, vol='GARCH', p=1, o=1, q=1, dist='StudentsT')
gjr_t_fit = model_gjr_t.fit(disp='off')

print(gjr_fit.summary())

# Extract parameters
omega_gjr_t = gjr_t_fit.params['omega']
alpha_gjr_t = gjr_t_fit.params['alpha[1]']
gamma_gjr_t = gjr_t_fit.params['gamma[1]']
beta_gjr_t = gjr_t_fit.params['beta[1]']

print(f"\n{'='*60}")
print("GJR-GARCH PARAMETERS")
print(f"{'='*60}")
print(f"ω (omega):                    {omega_gjr_t:.6f}")
print(f"α (alpha):                    {alpha_gjr_t:.6f}")
print(f"γ (gamma):                    {gamma_gjr_t:.6f}")
print(f"β (beta):                     {beta_gjr_t:.6f}")
print(f"\nTotal impact (negative):      {alpha_gjr_t + gamma_gjr_t:.6f}")
print(f"Asymmetry ratio:              {(alpha_gjr_t + gamma_gjr_t) / alpha_gjr_t:.3f}")

if gamma_gjr_t > 0:
    print(f"\n✓ Leverage effect confirmed: γ = {gamma_gjr_t:.6f} > 0")
    print(f"  Negative shocks increase volatility {((alpha_gjr_t + gamma_gjr_t) / alpha_gjr_t - 1) * 100:.1f}% more")

# Extract conditional volatility
gjr_t_vol = gjr_t_fit.conditional_volatility

In [ ]:
# Model comparison
print("\n" + "="*65)
print("MODEL COMPARISON")
print("="*65)
print(f"{'Model':<20} {'Log-Likelihood':>15} {'AIC':>12} {'BIC':>12}")
print("-"*65)
print(f"{'EWMA':<20} {ewma_fit.loglikelihood:>15.2f} {ewma_fit.aic:>12.2f} {ewma_fit.bic:>12.2f}")
print(f"{'GARCH(1,1)':<20} {garch_fit.loglikelihood:>15.2f} {garch_fit.aic:>12.2f} {garch_fit.bic:>12.2f}")
print(f"{'GJR-GARCH(1,1)':<20} {gjr_fit.loglikelihood:>15.2f} {gjr_fit.aic:>12.2f} {gjr_fit.bic:>12.2f}")
print(f"{'GARCH(1,1)-t':<20} {garch_t_fit.loglikelihood:>15.2f} {garch_t_fit.aic:>12.2f} {garch_t_fit.bic:>12.2f}")
print(f"{'GJR-GARCH(1,1)-t':<20} {gjr_t_fit.loglikelihood:>15.2f} {gjr_t_fit.aic:>12.2f} {gjr_t_fit.bic:>12.2f}")
print("-"*65)

## 4. Realized Volatility (RV)

### Oxford-Man Realized Volatility Dataset

Realized volatility is computed from high-frequency (intraday) data:

$$RV_t = \sum_{i=1}^{N} r_{t,i}^2$$

where $r_{t,i}$ are intraday returns.

**RV5**: Realized variance based on 5-minute returns

**Advantages over GARCH:**
- Uses high-frequency information
- Model-free measure of volatility
- More accurate proxy for true (latent) volatility

In [ ]:
rv5_vol = pd.read_csv('sp500rv5.csv', index_col=0, parse_dates=True)
rv5_vol.index = pd.to_datetime(rv5_vol.index, utc=True).tz_localize(None).normalize()

rv5_vol = rv5_vol['rv']

In [ ]:
# Align RV with returns data
rv5_aligned = rv5_vol.reindex(returns.index).dropna()

print(f"Aligned observations: {len(rv5_aligned)}")
print(f"Date range: {rv5_aligned.index[0]} to {rv5_aligned.index[-1]}")

In [ ]:
# Visualize RV5 vs GARCH volatility
# Align GARCH volatility with RV5 dates
garch_vol_aligned = garch_vol.reindex(rv5_aligned.index).dropna()
rv5_common = rv5_aligned.reindex(garch_vol_aligned.index)

fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Time series comparison
axes[0].plot(rv5_common.index, rv5_common.values, linewidth=0.8, 
             label='RV5 (Realized Volatility)', alpha=0.7, color='gray')
axes[0].plot(garch_vol_aligned.index, garch_vol_aligned.values, linewidth=1.5, 
             label='GARCH(1,1)', alpha=0.8, color='red')
axes[0].set_title('Realized Volatility (RV5) vs GARCH', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Volatility (%)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Scatter plot
axes[1].scatter(garch_vol_aligned.values, rv5_common.values, alpha=0.3, s=10)
axes[1].plot([0, garch_vol_aligned.max()], [0, garch_vol_aligned.max()], 
             'r--', linewidth=2, label='Perfect Forecast')
correlation = np.corrcoef(garch_vol_aligned.values, rv5_common.values)[0, 1]
axes[1].set_title(f'GARCH vs RV5 (Correlation = {correlation:.3f})', fontsize=12, fontweight='bold')
axes[1].set_xlabel('GARCH Volatility (%)')
axes[1].set_ylabel('Realized Volatility (%)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Forecast errors
errors = rv5_common.values - garch_vol_aligned.values
axes[2].plot(rv5_common.index, errors, linewidth=0.8, color='purple')
axes[2].axhline(y=0, color='red', linestyle='--', linewidth=0.5)
axes[2].set_title('GARCH Forecast Errors (RV5 - GARCH)', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Error (%)')
axes[2].set_xlabel('Date')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Error statistics
mse = mean_squared_error(rv5_common, garch_vol_aligned)
mae = mean_absolute_error(rv5_common, garch_vol_aligned)
print(f"\nGARCH Forecast Evaluation:")
print(f"MSE:  {mse:.6f}")
print(f"RMSE: {np.sqrt(mse):.6f}")
print(f"MAE:  {mae:.6f}")
print(f"Correlation: {correlation:.4f}")

## 5. HAR Model (Heterogeneous Autoregressive Model)

### Reference: Corsi (2009)

The HAR model (Corsi, 2009) is a simple but effective approach for forecasting realized volatility.

**Model Specification:**

$$RV_{t+1} = \beta_0 + \beta_D RV_t + \beta_W RV_t^{(w)} + \beta_M RV_t^{(m)} + \varepsilon_{t+1}$$

where:
- $RV_t$: Daily realized volatility
- $RV_t^{(w)} = \frac{1}{5}\sum_{i=0}^{4} RV_{t-i}$: Weekly average (5 days)
- $RV_t^{(m)} = \frac{1}{22}\sum_{i=0}^{21} RV_{t-i}$: Monthly average (22 days)

**Key Idea:**
- Different market participants have different trading horizons
- Daily traders, weekly traders, monthly traders
- This heterogeneity creates long memory in volatility

**Advantages:**
- Simple OLS estimation
- No distributional assumptions
- Often outperforms complex models
- Captures long memory properties

In [ ]:
# Prepare HAR model features
print("Preparing HAR model features...\n")

# Use the full SPX RV5 data
rv_df = pd.DataFrame({'rv': rv5_vol})

# Calculate weekly and monthly averages
rv_df['rv_daily'] = rv_df['rv']
rv_df['rv_weekly'] = rv_df['rv'].rolling(window=5, min_periods=5).mean()
rv_df['rv_monthly'] = rv_df['rv'].rolling(window=22, min_periods=22).mean()

# Shift features by 1 for forecasting (use t-1 to predict t)
rv_df['rv_daily_lag'] = rv_df['rv_daily'].shift(1)
rv_df['rv_weekly_lag'] = rv_df['rv_weekly'].shift(1)
rv_df['rv_monthly_lag'] = rv_df['rv_monthly'].shift(1)

# Target variable (RV at time t)
rv_df['rv_target'] = rv_df['rv']

# Drop NaN values
rv_df_clean = rv_df.dropna()

print(f"HAR model data prepared:")
print(f"Total observations: {len(rv_df_clean)}")
print(f"\nFirst few rows:")
print(rv_df_clean[['rv_target', 'rv_daily_lag', 'rv_weekly_lag', 'rv_monthly_lag']].head(10))

In [ ]:
train_df = rv_df_clean.copy()

print(f"Training set: {len(train_df)} observations ({train_df.index[0]} to {train_df.index[-1]})")

In [ ]:
# Estimate HAR model using OLS
print("\nEstimating HAR model using OLS...\n")

# Prepare features and target
X_train = train_df[['rv_daily_lag', 'rv_weekly_lag', 'rv_monthly_lag']]
y_train = train_df['rv_target']


# Fit HAR model
har_model = LinearRegression()
har_model.fit(X_train, y_train)

# Print coefficients
print("="*60)
print("HAR MODEL ESTIMATION RESULTS")
print("="*60)
print(f"\nModel: RV(t) = β₀ + βD·RV(t-1) + βW·RV_weekly(t-1) + βM·RV_monthly(t-1)\n")
print(f"Intercept (β₀):  {har_model.intercept_:>10.6f}")
print(f"Daily (βD):      {har_model.coef_[0]:>10.6f}")
print(f"Weekly (βW):     {har_model.coef_[1]:>10.6f}")
print(f"Monthly (βM):    {har_model.coef_[2]:>10.6f}")
print(f"\nSum of coefficients: {har_model.coef_.sum():.6f}")

# In-sample fit
y_train_pred = har_model.predict(X_train)
r2_train = r2_score(y_train, y_train_pred)
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

print(f"\nIn-sample performance:")
print(f"  R²:   {r2_train:.6f}")
print(f"  MSE:  {mse_train:.6f}")
print(f"  RMSE: {np.sqrt(mse_train):.6f}")
print(f"  MAE:  {mae_train:.6f}")

In [ ]:
# Visualize HAR model forecasts
fig, ax = plt.subplots(figsize=(10, 4))

# In-sample fit
ax.plot(train_df.index, y_train.values, linewidth=0.8, 
             label='Realized RV5', alpha=0.6, color='black')
ax.plot(train_df.index, y_train_pred, linewidth=1, 
             label='HAR Forecast', alpha=0.8, color='blue')
ax.set_title(f'In-Sample Fit (R² = {r2_train:.4f})', fontsize=12, fontweight='bold')
ax.set_ylabel('Volatility (%)')
ax.legend()
ax.grid(True, alpha=0.3)


plt.tight_layout()
plt.show()

## 6. HAR-X Model with VIX

### HAR with Exogenous Variables

The HAR-X model extends the basic HAR by including exogenous variables that may contain additional information about future volatility.

**Model Specification:**

$$RV_{t+1} = \beta_0 + \beta_D RV_t + \beta_W RV_t^{(w)} + \beta_M RV_t^{(m)} + \beta_{VIX} VIX_t + \varepsilon_{t+1}$$

**VIX (CBOE Volatility Index):**
- Implied volatility from S&P 500 options
- Forward-looking measure of market expectations
- Often called the "fear index"
- May contain information not captured by historical RV


In [ ]:
# Connect to WRDS and download CBOE VIX
import wrds

# Connect to WRDS (you might have to confirm 2FA)
db = wrds.Connection(wrds_username=None)

# Download CBOE VIX index data from CBOE
# Date range: 2000-01-03 to 2022-06-28 (matching RV5 data)
query = """
SELECT date, vix
FROM cboe.cboe
WHERE date >= '2000-01-03' 
  AND date <= '2022-06-28'
ORDER BY date
"""

print("Downloading CBOE VIX data from WRDS/CBOE...")
vix = db.raw_sql(query)
db.close()

# Process data
vix['date'] = pd.to_datetime(vix['date'])
vix = vix.set_index('date')
vix = vix[~vix.index.duplicated(keep='first')]
vix = vix['vix'].dropna()

print(f"\nData loaded: {len(vix)} observations")
print(f"Date range: {vix.index[0]} to {vix.index[-1]}")
print(f"\nBasic statistics:")
print(vix.describe())

In [ ]:
# # Load VIX data from CBOE (alternative)
# print("Loading VIX data from CBOE...\n")

# # Fetch VIX historical data
# vix_url = 'https://cdn.cboe.com/api/global/us_indices/daily_prices/VIX_History.csv'
# vix_data = pd.read_csv(vix_url)

# # Process VIX data
# vix_data.columns = [col.strip().upper() for col in vix_data.columns]
# vix_data['DATE'] = pd.to_datetime(vix_data['DATE'])
# vix_data = vix_data.set_index('DATE')

# # Select closing VIX
# vix = vix_data['CLOSE'].copy()
# vix.name = 'vix'

# print(f"VIX data loaded successfully!")
# print(f"Total observations: {len(vix)}")
# print(f"Date range: {vix.index.min()} to {vix.index.max()}")
# print(f"\nFirst few rows:")
# print(vix.head())
# print(f"\nVIX statistics:")
# print(vix.describe())

In [ ]:
# Prepare HAR-X model with VIX
print("\nPreparing HAR-X model features...\n")

# Merge VIX with RV data
rv_df_harx = rv_df_clean.copy()
rv_df_harx['vix'] = vix.reindex(rv_df_harx.index)

# Shift VIX by 1 (use VIX at t-1 to predict RV at t)
rv_df_harx['vix_lag'] = rv_df_harx['vix'].shift(1)

# Drop NaN values
rv_df_harx = rv_df_harx.dropna()

print(f"HAR-X model data prepared:")
print(f"Total observations: {len(rv_df_harx)}")
print(f"\nFirst few rows:")
print(rv_df_harx[['rv_target', 'rv_daily_lag', 'rv_weekly_lag', 'rv_monthly_lag', 'vix_lag']].head())

In [ ]:
train_df_harx = rv_df_harx.copy()

print(f"Training set: {len(train_df_harx)} observations ({train_df_harx.index[0]} to {train_df_harx.index[-1]})")

In [ ]:
# Estimate HAR-X model using OLS
print("\nEstimating HAR-X model using OLS...\n")

# Prepare features and target
X_train_harx = train_df_harx[['rv_daily_lag', 'rv_weekly_lag', 'rv_monthly_lag', 'vix_lag']]
y_train_harx = train_df_harx['rv_target']


# Fit HAR-X model
harx_model = LinearRegression()
harx_model.fit(X_train_harx, y_train_harx)

# Print coefficients
print("="*60)
print("HAR-X MODEL ESTIMATION RESULTS")
print("="*60)
print(f"\nModel: RV(t) = β₀ + βD·RV(t-1) + βW·RV_weekly(t-1) + βM·RV_monthly(t-1) + βVIX·VIX(t-1)\n")
print(f"Intercept (β₀):  {harx_model.intercept_:>10.6f}")
print(f"Daily (βD):      {harx_model.coef_[0]:>10.6f}")
print(f"Weekly (βW):     {harx_model.coef_[1]:>10.6f}")
print(f"Monthly (βM):    {harx_model.coef_[2]:>10.6f}")
print(f"VIX (βVIX):      {harx_model.coef_[3]:>10.6f}")

# In-sample fit
y_train_pred_harx = harx_model.predict(X_train_harx)
r2_train_harx = r2_score(y_train_harx, y_train_pred_harx)
mse_train_harx = mean_squared_error(y_train_harx, y_train_pred_harx)
mae_train_harx = mean_absolute_error(y_train_harx, y_train_pred_harx)

print(f"\nIn-sample performance:")
print(f"  R²:   {r2_train_harx:.6f}")
print(f"  MSE:  {mse_train_harx:.6f}")
print(f"  RMSE: {np.sqrt(mse_train_harx):.6f}")
print(f"  MAE:  {mae_train_harx:.6f}")

In [ ]:
# Visualize HAR-VIX model 
fig, ax = plt.subplots(figsize=(10, 4))

# In-sample fit
ax.plot(train_df_harx.index, y_train_harx.values, linewidth=0.8, 
             label='Realized RV5', alpha=0.6, color='black')
ax.plot(train_df_harx.index, y_train_pred_harx, linewidth=1, 
             label='HAR-VIX Forecast', alpha=0.8, color='blue')
ax.set_title(f'In-Sample Fit (R² = {r2_train_harx:.4f})', fontsize=12, fontweight='bold')
ax.set_ylabel('Volatility (%)')
ax.legend()
ax.grid(True, alpha=0.3)


plt.tight_layout()
plt.show()